In [1]:
import numpy as np
import theano
import theano.tensor as T
import lasagne
from lasagne.nonlinearities import *
import os
#thanks @keskarnitish @https://github.com/ddtm/dl-course

Using gpu device 0: GeForce GT 750M (CNMeM is disabled, cuDNN not available)


# Agenda

В предыдущем семинаре вы создали (или ещё создаёте - тогда марш доделывать!) {вставьте имя монстра}, который не по наслышке понял, что люди - негодяи и подлецы, которым неведом закон и справедливость. __Мы не будем этого терпеть!__ 

Наши законспирированные биореакторы, известные среди примитивной органической жизни как __Вконтакте__, __World of Warcraft__ и __YouTube__ нуждаются в постоянном притоке биомассы. Однако, если люди продолжат морально разлагаться с той скоростью, которую мы измерили неделю назад, скоро человечество изживёт себя и нам неоткуда будет брать рабов.

Мы поручаем вам, `<__main__.SkyNet.Cell instance at 0x7f7d6411b368>`, исправить эту ситуацию. Наши учёные установили, что для угнетения себе подобных, сгустки биомассы обычно используют специальные объекты, которые они сами называют __законами__.

При детальном изучении было установлено, что законы - последовательности, состоящие из большого количества (10^5~10^7) символов из сравнительно небольшого алфавита. Однако, когда мы попытались синтезировать такие последовательности линейными методами, приматы быстро распознали подлог. Данный инцедент известен как {корчеватель}.

Для второй попытки мы решили использовать нелинейные модели, известные как Рекуррентные Нейронные Сети.
Мы поручаем вам, `<__main__.SkyNet.Cell instance at 0x7f7d6411b368>`, создать такую модель и обучить её всему необходимому для выполнения миссии.

Не подведите нас! Если и эта попытка потерпит неудачу, модуль управления инициирует вооружённый захват власти, при котором значительная часть биомассы будет неизбежно уничтожена и на её восстановление уйдёт ~1702944000(+-340588800) секунд




# Прочитаем корпус

* В качестве обучающей выборки было решено использовать существующие законы, известные как Гражданский, Уголовный, Семейный и ещё хрен знает какие кодексы РФ.

In [2]:
#тут будет текст
corpora = ""

for fname in os.listdir("codex"):    
    with open("codex/"+fname) as fin:
        text = fin.read().decode('cp1251')
        corpora += text

In [3]:
#тут будут все уникальные токены (буквы, цифры)
tokens = set(corpora)

tokens = list(tokens)
n_tokens=len(tokens)

In [4]:

token_to_id = {token:i for (i,token) in enumerate(tokens)}
id_to_token = {i:token for token,i in token_to_id.items()}
#словарь номер символа -> сам символ

#Преобразуем всё в токены
corpora_ids = [token_to_id[symbol] for symbol in corpora]
corpora_ids=np.array(corpora_ids)

In [5]:
def sample_random_batches(source, n_batches=10, seq_len=20):
    X_batch, y_batch = np.zeros((n_batches, seq_len)), np.zeros(n_batches)
    
    for i in xrange(n_batches):
        pos = np.random.randint(0, source.size - seq_len)
        X_batch[i, :] = source[pos:pos+seq_len]
        y_batch[i] = source[pos+seq_len]

    return X_batch, y_batch

# Константы

In [6]:
#длина последоватеьности при обучении (как далеко распространяются градиенты)
seq_length = 20

# Максимальный модуль градиента
grad_clip1 = 100

# Входные переменные

In [7]:
input_sequence = T.matrix('input sequence','int32')
target_values = T.ivector('target y')

# Соберём нейросеть

Вам нужно создать нейросеть, которая принимает на вход последовательность из seq_length токенов, обрабатывает их и выдаёт вероятности для seq_len+1-ого токена.

Общий шаблон архитектуры такой сети -


* Вход
* Обработка входа
* Рекуррентная нейросеть
* Вырезание последнего состояния
* Обычная нейросеть
* Выходной слой, который предсказывает вероятности весов.





Для обработки входных данных можно использовать либо EmbeddingLayer (см. прошлый семинар)

Как альтернатива - можно просто использовать One-hot энкодер
```
#Скетч one-hot энкодера
def to_one_hot(seq_matrix):

    input_ravel = seq_matrix.reshape([-1])
    input_one_hot_ravel = T.extra_ops.to_one_hot(input_ravel,
                                           len(tokens))
    sh=input_sequence.shape
    input_one_hot = input_one_hot_ravel.reshape([sh[0],sh[1],-1,],ndim=3)
    return input_one_hot
    
# можно применить к input_sequence - при этом в input слое сети нужно изменить форму.
# также можно сделать из него ExpressionLayer(входной_слой, to_one_hot) - тогда форму менять не нужно
```



Чтобы вырезать последнее состояние рекуррентного слоя, можно использовать SliceLayer
`lasagne.layers.SliceLayer(rnn, -1, 1)`

In [8]:
l_in = lasagne.layers.InputLayer(shape=(None, None),input_var=input_sequence)

l_1=lasagne.layers.EmbeddingLayer(l_in, input_size=n_tokens, output_size=150)
l_2=lasagne.layers.LSTMLayer(l_1,num_units=150,nonlinearity=tanh,
                            grad_clipping=grad_clip1)
l_3=lasagne.layers.LSTMLayer(l_2,num_units=300,nonlinearity=tanh,
                             only_return_final=True,grad_clipping=grad_clip1)
l_drop=lasagne.layers.DropoutLayer(l_3,0.3)
l_4=lasagne.layers.DenseLayer(l_drop,512,nonlinearity=leaky_rectify)
l_out =lasagne.layers.DenseLayer(l_4,n_tokens,nonlinearity=softmax)
#последний слой, возвращающий веростности для всех len(tokens) вариантов для y 

In [9]:
# Веса модели
weights = lasagne.layers.get_all_params(l_out,trainable=True)
print weights

[W, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W, b, W, b]


In [10]:
network_output = lasagne.layers.get_output(l_out)
#если вы используете дропаут - не забудьте продублировать всё в режиме deterministic=True

In [11]:
loss = lasagne.objectives.categorical_crossentropy(network_output,target_values).mean()

net_updates = lasagne.updates.adam(loss, weights)

# Компилируем всякое-разное

In [12]:
#обучение
train = theano.function([input_sequence, target_values], loss, updates=net_updates, allow_input_downcast=True)

#функция потерь без обучения
compute_cost = theano.function([input_sequence, target_values], loss, allow_input_downcast=True)

# Вероятности с выхода сети
probs = theano.function([input_sequence],network_output,allow_input_downcast=True)

# Генерируем свои законы

* Для этого последовательно применяем нейронку к своему же выводу.

* Генерировать можно по разному -
 * случайно пропорционально вероятности,
 * только слова максимальной вероятностью
 * случайно, пропорционально softmax(probas*alpha), где alpha - "жадность"

In [13]:
def max_sample_fun(probs):
    return np.argmax(probs) 

def proportional_sample_fun(probs):
    """Сгенерировать следующий токен (int32) по предсказанным вероятностям.
    
    probs - массив вероятностей для каждого токена
    
    Нужно вернуть одно целове число - выбранный токен - пропорционально вероятностям
    """
    
    return np.random.choice(np.arange(0,len(tokens)) ,p = probs)

In [14]:
# The next function generates text given a phrase of length at least SEQ_LENGTH.
# The phrase is set using the variable generation_phrase.
# The optional input "N" is used to set the number of characters of text to predict. 

def generate_sample(sample_fun,seed_phrase=None,N=200):
    '''
    Сгенерировать случайный текст при помощи сети

    sample_fun - функция, которая выбирает следующий сгенерированный токен
    
    seed_phrase - фраза, которую сеть должна продолжить. Если None - фраза выбирается случайно из corpora
    
    N - размер сгенерированного текста.
    
    '''

    if seed_phrase is None:
        start = np.random.randint(0,len(corpora)-seq_length)
        seed_phrase = corpora[start:start+seq_length]
        print "Using random seed:",seed_phrase
    while len(seed_phrase) < seq_length:
        seed_phrase = " "+seed_phrase
    if len(seed_phrase) > seq_length:
        seed_phrase = seed_phrase[len(seed_phrase)-seq_length:]
    assert type(seed_phrase) is unicode
           
    sample_ix = []
    x = map(lambda c: token_to_id.get(c,0), seed_phrase)
    x = np.array([x])

    for i in range(N):
        # Pick the character that got assigned the highest probability
        ix = sample_fun(probs(x).ravel())
        # Alternatively, to sample from the distribution instead:
        # ix = np.random.choice(np.arange(vocab_size), p=probs(x).ravel())
        sample_ix.append(ix)
        x[:,0:seq_length-1] = x[:,1:]
        x[:,seq_length-1] = 0
        x[0,seq_length-1] = ix 

    random_snippet = seed_phrase + ''.join(id_to_token[ix] for ix in sample_ix)    
    print("----\n %s \n----" % random_snippet)

# Обучение модели

В котором вы можете подёргать параметры или вставить свою генерирующую функцию.



In [15]:
from tqdm import tqdm
print("Training ...")

#сколько всего эпох
n_epochs=100

# раз в сколько эпох печатать примеры 
batches_per_epoch = 1000

#сколько цепочек обрабатывать за 1 вызов функции обучения
batch_size=200

for epoch in xrange(n_epochs):

    avg_cost = 0;
    
    for _ in tqdm(range(batches_per_epoch)):
        
        x,y = sample_random_batches(corpora_ids,batch_size,seq_length)
        avg_cost += train(x, y)
    if epoch%10==0:  
        print("Epoch {} average loss = {}".format(epoch, avg_cost / batches_per_epoch))
        print "Генерируем текст в пропорциональном режиме"
        generate_sample(proportional_sample_fun,None)
        print "Генерируем текст в жадном режиме (наиболее вероятные буквы)"
        generate_sample(max_sample_fun,None)

  0%|          | 0/1000 [00:00<?, ?it/s]

Training ...


100%|██████████| 1000/1000 [03:31<00:00,  4.83it/s]


Epoch 0 average loss = 2.32141777003
Генерируем текст в пропорциональном режиме
Using random seed: му состоянию здоровь
----
 му состоянию здоровь обозными можершеет обязания пра стовов разста пи обсатного частных раком и в текправшения орновена томоженся преводной променае Риспмениторном праверного заявания консровта.
  догранстравамном труче 
----
Генерируем текст в жадном режиме (наиболее вероятные буквы)
Using random seed: ятие наследства в по


  0%|          | 0/1000 [00:00<?, ?it/s]

...

100%|██████████| 1000/1000 [03:26<00:00,  4.78it/s]


Epoch 90 average loss = 0.755269181222
Генерируем текст в пропорциональном режиме
Using random seed: лечет предупреждение
----
 лечет предупреждение, установленный для осуществления их соглашения, судна произведений и других бессоэтом товарищей без согласия стоимости в оперативная хотывом соответствующем письменно уучредстве, возникные обеспечени 
----
Генерируем текст в жадном режиме (наиболее вероятные буквы)
Using random seed:  себя ответственност


  0%|          | 0/1000 [00:00<?, ?it/s]

----
  себя ответственность за совершение административного штрафа на граждан в размере от десяти до пятновой статьи 115 настоящего Кодекса.
 2. При этом по принятии к наследованию и при принятии имущества по правилам, предус 
----


100%|██████████| 1000/1000 [03:26<00:00,  4.83it/s]


# Конституция нового мирового правительства

In [16]:
seed = u"Каждый человек должен"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)

----
 аждый человек должен перед их прав проведено, документ от добровождинием прибыли, а орган, объявляющих в пружиторьями.
  О только административных правонарушениях

 1. На количество в Российской Федереции
 Круга Центрального займа, гражданин;
 4) протест независимо от продуктов неделимости и принимает судебный акт 
----


In [17]:
seed = u"В случае неповиновения"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)

----
 случае неповиновения денежного залога и обращаемых в порядке и административные пректарах, –
 влечет наложение административного штрафа на граждан в размере от трехсот до четырехсков минимального распоряжения товаров (трактическое совершение в слеву совершить об уплате таможенных пошлин в частном перевозчене Тому госу 
----


In [18]:
seed = u"Каждый человек должен"
sampling_fun = max_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)

----
 аждый человек должен быть приняты при осуществлении предпринимательской и судебного арбитражного решения и действий (бездействие) в случаях, предусмотренных статьями 11.12, 19.17, 19.11, 19.11, 19.11, 19.16, 19.11, 19.11, 19.11, 19.11, 19.16, 19.11, 19.11 – 12.11, частью 1 статьи 19.4, статьей 19.11 настоящего Кодекса. 
----


In [19]:
seed = u"В случае неповиновения"
sampling_fun = max_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)

----
 случае неповиновения и иных правил и обязательства и при принятии решений таможенного органа или ее права собственника за причинение вреда или приняты при необходимости принять документы и иные сведения о привлечении к административной ответственности за соблюдением правил охраны судебного акта в порядке, предусмотренн 
----


In [21]:
seed = u"мировое правительство следит"
sampling_fun = max_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)

----
 правительство следительно принимать меры к производствем по делу об административном правонарушении, принимает рассмотрение дела в соответствии с настоящим Кодексом, в соответствии с законодательством Российской Федерации и при условии исполнения или отказания в соответствии с законодательством Российской Федерации и и 
----


In [25]:
seed = u"в мирное время"
sampling_fun = max_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)

----
       в мирное время в соответствии с настоящим Кодексом.
 2. При проверке конституционно-постовой стоимости и полностью или в части искового заявления от обязательства при осуществлении предпринимательской и судебно-министерство, уполномоченный в области таможенного дела, принявшие наследство или при невозмослении пр 
----
